In [49]:
#Importing needed ML library

In [50]:
import keras

In [51]:
#mnist has the data, sequential is the model type, layers are the type of NN layers

In [52]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [53]:
#define a tuple for training and testing data. Both are the dataset of handwritten digits

In [54]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape) #show size of dataset

(60000, 28, 28) (60000,)


In [55]:
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)#changing the size to include one extra dimension of size 1?
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)#defining 3-tuple as 28 by 28 by 1

In [56]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes) #y_train is the x_train put through model
y_test = keras.utils.to_categorical(y_test, num_classes) #y_test is x_test put through model
x_train = x_train.astype('float32') #casting to float
x_test = x_test.astype('float32') #casting to float
x_train /= 255 #dividing all values in x_train by 255?
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [57]:
batch_size = 128
epochs = 3
# #This is going to be our model. It is a sequential NN
model = Sequential() 
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
# model=Sequential()
# model.add(Dense(512, activation='relu', input_shape=(28 * 28,)))
# model.add(Dense(10, activation='softmax'))
# model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])


In [58]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))#train model
print("The model has successfully trained")
model.save('mnist.h5') #saving file
print("Saving the model as mnist.h5")

Epoch 1/3
469/469 [==============================] - 53s 107ms/step - loss: 0.1841 - accuracy: 0.9441 - val_loss: 0.0445 - val_accuracy: 0.9859
Epoch 2/3
469/469 [==============================] - 50s 107ms/step - loss: 0.0619 - accuracy: 0.9809 - val_loss: 0.0355 - val_accuracy: 0.9883
Epoch 3/3
469/469 [==============================] - 51s 109ms/step - loss: 0.0449 - accuracy: 0.9858 - val_loss: 0.0304 - val_accuracy: 0.9905
The model has successfully trained
Saving the model as mnist.h5


In [59]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.030430709943175316
Test accuracy: 0.9904999732971191


In [60]:
from keras.models import load_model
from PIL import ImageGrab, ImageOps, Image
import numpy as np

model = load_model('mnist.h5')
img = Image.open('five.PNG')
img.show()
img = img.resize((28,28))
img = img.convert('L')
img = np.array(img)
img = img.reshape(1,28,28,1)
img = img/255.0
res = model.predict([img])[0]
print(np.argmax(res))
print(max(res))

model = load_model('mnist.h5')
img = Image.open('two.png')
img.show()
img = img.resize((28,28))
img = img.convert('L')
img = np.array(img)
img = img.reshape(1,28,28,1)
img = img/255.0
res = model.predict([img])[0]
print(np.argmax(res))
print(max(res))

model = load_model('mnist.h5')
img = Image.open('test.PNG')
img.show()
img = img.resize((28,28))
img = img.convert('L')
img = np.array(img)
img = img.reshape(1,28,28,1)
img = img/255.0
res = model.predict([img])[0]
print(np.argmax(res))
print(max(res))


1/1 [==============================] - 2s 2s/step
5
0.945367
1/1 [==============================] - 2s 2s/step
2
0.99967146
1/1 [==============================] - 2s 2s/step
6
0.9999987


In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, ImageOps
import numpy as np
model = load_model('mnist.h5')
def predict_digit(img):
    #resize image to 28×28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    #invert colors so background is black, number is white
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        global correct
        global total
        correct  = 1
        total = 1
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")      
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
#         self.label1 = tk.Label(self, text=correct, font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.correct_btn = tk.Button(self, text = "Correct", command = self.add_correct)
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
#         self.label1.grid(row=0, column=2,pady=2, padx=2)

        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.correct_btn.grid(row=1, column=2, pady=2)
#         self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def add_correct(self):
        global correct
        correct = correct + 1
        print(correct)

    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas

#         rect[0] = win32gui.GetWindowRect(HWND)[0]+10 # get the coordinate of the canvas
#         rect[1] = win32gui.GetWindowRect(HWND)[1]+10 # get the coordinate of the canvas
#         rect[2] = win32gui.GetWindowRect(HWND)[2]+10 # get the coordinate of the canvas
#         rect[3] = win32gui.GetWindowRect(HWND)[3]+10 # get the coordinate of the canvas

        im = ImageGrab.grab(rect)
        im = ImageOps.invert(im)
        print(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=6
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()

(0, 25, 304, 329)
1/1 [==============================] - 2s 2s/step
(0, 25, 304, 329)
1/1 [==============================] - 2s 2s/step
